# **Replacing missing values with a value at the end of the distribution**

In this work, we will replace missing values by a value at the end of the distribution, estimated with the Gaussian approximation or the inter-quantal range proximity rule, using pandas and Feature-Engine.

In [1]:
pip install feature-engine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 6.2 MB/s eta 0:00:00


In [2]:
import pandas as pd

# to split the datasets
from sklearn.model_selection import train_test_split

# to impute missing data with feature-engine
from feature_engine.imputation import EndTailImputer

# **End tail imputation with pandas**

In [3]:
# load data
data = pd.read_csv('creditApprovalUCI.csv')
data.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,1
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,1
2,a,24.50,NaN,u,g,q,h,NaN,NaN,NaN,0,f,g,280.0,824,1
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,1
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,1


In [4]:
# let's separate into training and testing set

X_train, X_test, y_train, y_test = train_test_split(
    data.drop('A16', axis=1), data['A16'], test_size=0.3, random_state=0)

X_train.shape, X_test.shape

((483, 15), (207, 15))

In [5]:
# find the percentage of missing data per variable

X_train.isnull().mean()

A1     0.008282
A2     0.022774
A3     0.140787
A4     0.008282
A5     0.008282
A6     0.008282
A7     0.008282
A8     0.140787
A9     0.140787
A10    0.140787
A11    0.000000
A12    0.000000
A13    0.000000
A14    0.014493
A15    0.000000
dtype: float64

In [6]:
# replace NA in indicated numerical variables
# using inter-quantal range proximity rule

for var in ['A2', 'A3', 'A8', 'A11', 'A15']:

    IQR = X_train[var].quantile(0.75) - X_train[var].quantile(0.25)
    value = X_train[var].quantile(0.75) + 1.5 * IQR

    X_train[var] = X_train[var].fillna(value)
    X_test[var] = X_test[var].fillna(value)

In [7]:
# check absence of missing values
X_train[['A2', 'A3', 'A8', 'A11', 'A15']].isnull().sum()

A2     0
A3     0
A8     0
A11    0
A15    0
dtype: int64

# **End tail imputation with Feature Engine**

In [8]:
# let's separate into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(
    data.drop('A16', axis=1), data['A16'], test_size=0.3, random_state=0)

In [12]:
# let's create a median imputer

imputer = EndTailImputer(imputation_method='gaussian', tail='right', fold=3,
                         variables=['A2', 'A3', 'A8', 'A11', 'A15'])

imputer.fit(X_train)

EndTailImputer(variables=['A2', 'A3', 'A8', 'A11', 'A15'])

In [13]:
# dictionary with the mappings for each variable
imputer.imputer_dict_

{'A2': 68.35771260807589,
 'A3': 19.98993346546277,
 'A8': 12.418567732660225,
 'A11': 18.320547522636247,
 'A15': 12740.850618383225}

In [14]:
# transform the data
X_train = imputer.transform(X_train)
X_test = imputer.transform(X_test)

In [15]:
# check that null values were replaced
X_train[['A2', 'A3', 'A8', 'A11', 'A15']].isnull().mean()

A2     0.0
A3     0.0
A8     0.0
A11    0.0
A15    0.0
dtype: float64